In [ ]:
pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib azure-cognitiveservices-vision-computervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 7.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
import os
import numpy as np
from PIL import Image
import io
import csv

# Montar Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Configuración de Azure Computer Vision API
subscription_key = "5VScBomj0F99NCHIUveba920ULiJzElLv3BBQerUQMvRwbkdPyssJQQJ99BAACYeBjFXJ3w3AAAFACOGE5hR" #Enter a valid key
endpoint = "https://prueba-vision.cognitiveservices.azure.com/" #Enter a endpoint
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

# Rutas a las carpetas en Google Drive (después de montar)
carpeta_origen = '/content/drive/MyDrive/Proyecto EOI/Fotos/Fotos a tratar'
carpeta_destino = '/content/drive/MyDrive/Proyecto EOI/Fotos/Fotos tratadas'
csv_file = '/content/drive/MyDrive/Proyecto EOI/Fotos/analisis_imagenes.csv'

# Inicializar el archivo CSV (si no existe)
def initialize_csv():
    if not os.path.exists(csv_file):
        with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(["Nombre de archivo", "URL de Imagen", "Número de bicicletas", "Descripción", "Modelo bicicletas",])
initialize_csv()

def analizar_imagen(ruta_imagen):
    """Analiza una imagen, detecta bicicletas y su estado y guarda la informacion en CSV."""

    try:
        # Abrir la imagen con Pillow
        img = Image.open(ruta_imagen)

        # Redimensionar la imagen si es demasiado grande
        max_dimension = 1024
        if img.width > max_dimension or img.height > max_dimension:
            img.thumbnail((max_dimension, max_dimension))

        # Guardar la imagen redimensionada en un buffer in-memory
        buffered = io.BytesIO()
        img.save(buffered, format=img.format)
        buffered.seek(0)

        # Analizar la imagen con Azure Computer Vision (usar el buffer)
        image_analysis = computervision_client.analyze_image_in_stream(
            buffered, visual_features=[VisualFeatureTypes.objects, VisualFeatureTypes.description]
        )
        descripcion = image_analysis.description.captions[0].text if image_analysis.description.captions else "No hay descripción disponible"
    except Exception as e:
        print(f"Error al procesar la imagen {ruta_imagen}: {e}")
        return

    # Contar bicicletas
    bicicletas = [obj for obj in image_analysis.objects if obj.object_property == "bicycle"]
    num_bicicletas = len(bicicletas)


    modelos_similares = "No se pudo determinar"
    # Intentar determinar si son iguales (heurística simple)
    if num_bicicletas > 1:
        rectangulos = [obj.rectangle for obj in bicicletas]
        areas = [(r.w * r.h) for r in rectangulos]
        desviacion_estandar = np.std(areas)
        if desviacion_estandar < 0.7 * np.mean(areas) :
             modelos_similares = "Las bicicletas parecen ser del mismo modelo."
        else:
             modelos_similares = "Las bicicletas parecen ser de diferentes modelos."
    elif num_bicicletas == 1:
        modelos_similares = "Solo hay un modelo de bicicleta detectada."
    else:
        modelos_similares = "No se detectaron bicicletas."

    # Mover la imagen
    nombre_archivo = os.path.basename(ruta_imagen)
    nueva_ruta = os.path.join(carpeta_destino, nombre_archivo)
    os.rename(ruta_imagen, nueva_ruta)

    # Guardar la información en el CSV
    with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([nombre_archivo, nueva_ruta, num_bicicletas, descripcion, modelos_similares])

    print(f'Analisis imagen: {nombre_archivo}, numero de bicis: {num_bicicletas}, descripcion: {descripcion}, modelos: {modelos_similares}')



# Iterar sobre las imágenes en la carpeta de origen
for filename in os.listdir(carpeta_origen):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        ruta_imagen = os.path.join(carpeta_origen, filename)
        print(f'Analizando imagen: {filename}')
        analizar_imagen(ruta_imagen)

print("Análisis completado. Los datos se han guardado en:", csv_file)

Analizando imagen: descarga (3) - Zésar Vidal.jpg
Analisis imagen: descarga (3) - Zésar Vidal.jpg, numero de bicis: 2, descripcion: a man and woman riding a bicycle, modelos: Las bicicletas parecen ser del mismo modelo.
Analizando imagen: descarga (2) - Zésar Vidal.jpg
Analisis imagen: descarga (2) - Zésar Vidal.jpg, numero de bicis: 1, descripcion: a bicycle parked on the side of a road, modelos: Solo hay un modelo de bicicleta detectada.
Analizando imagen: descarga (1) - Zésar Vidal.jpg
Analisis imagen: descarga (1) - Zésar Vidal.jpg, numero de bicis: 1, descripcion: a person riding a bicycle, modelos: Solo hay un modelo de bicicleta detectada.
Análisis completado. Los datos se han guardado en: /content/drive/MyDrive/Proyecto EOI/Fotos/analisis_imagenes.csv
